В изысканиях опирался на данную статью: https://medium.com/radon-dev/als-implicit-collaborative-filtering-5ed653ba39fe
Предварительно прослушал про ALS https://www.youtube.com/watch?v=NlNLtPqlCK0

In [1]:
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random

from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings("ignore")

import implicit

### Load Data

In [2]:
train = pd.read_csv('input/train_2.csv')
test = pd.read_csv('input/test_2.csv')
user_features = pd.read_csv('input/user-features_2.csv')
item_features = pd.read_csv('input/item-features_2.csv')

### Preprocessing

In [3]:
train_full = train.merge(user_features, how='left', left_on='user_id', right_on='user_id')
train_full = train_full.merge(item_features, how='left', left_on='item_id', right_on='item_id')

### Feature Engineers

In [4]:
# convert to datetime trains

train_full['created'] = pd.to_datetime(train_full['timestamp'],unit='s')

# create new date features

train_full['created_month'] = train_full['created'].dt.month
train_full['created_day'] = train_full['created'].dt.day
train_full['created_dayofweek'] = train_full['created'].dt.dayofweek
train_full['created_hour'] = train_full['created'].dt.round('H').dt.hour

def f(x):
    if (x > 4) and (x <= 8):
        return 'Early Morning'
    elif (x > 8) and (x <= 12 ):
        return 'Morning'
    elif (x > 12) and (x <= 16):
        return'Noon'
    elif (x > 16) and (x <= 20) :
        return 'Eve'
    elif (x > 20) and (x <= 24):
        return'Night'
    elif (x <= 4):
        return'Late Night'
    
    
train_full['part_of_day'] = train_full['created_hour'].apply(f)

# train_full = train_full.drop(['created', 'timestamp'], axis=1)

# для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
train_full = pd.get_dummies(train_full, columns=[ 'part_of_day',], dummy_na=True)

# удалю окончательно колонку даты и времени нажатия на баннер
train_full = train_full.drop(['created'], axis = 1)

if train_full.part_of_day_nan.isnull().sum() == 0:
    train_full = train_full.drop(['part_of_day_nan'], axis = 1)

In [5]:
# convert to datetime test

test['created'] = pd.to_datetime(test['timestamp'],unit='s')

# create new date features

test['created_month'] = test['created'].dt.month
test['created_day'] = test['created'].dt.day
test['created_dayofweek'] = test['created'].dt.dayofweek
test['created_hour'] = test['created'].dt.round('H').dt.hour

def f(x):
    if (x > 4) and (x <= 8):
        return 'Early Morning'
    elif (x > 8) and (x <= 12 ):
        return 'Morning'
    elif (x > 12) and (x <= 16):
        return'Noon'
    elif (x > 16) and (x <= 20) :
        return 'Eve'
    elif (x > 20) and (x <= 24):
        return'Night'
    elif (x <= 4):
        return'Late Night'
    
    
test['part_of_day'] = test['created_hour'].apply(f)

# train_full = train_full.drop(['created', 'timestamp'], axis=1)

# для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
test = pd.get_dummies(test, columns=[ 'part_of_day',], dummy_na=True)

# удалю окончательно колонку даты и времени нажатия на баннер
test = test.drop(['created'], axis = 1)

if test.part_of_day_nan.isnull().sum() == 0:
    test = test.drop(['part_of_day_nan'], axis = 1)

In [6]:
banner_pivot=train_full.pivot(index='user_id',columns='item_id',values='like')
banner_pivot=banner_pivot.fillna(0)
banner_pivot.shape

(497, 444)

In [7]:
sparse_matrix = train_full.pivot(index='user_id',columns='item_id',values='like')
sparse_matrix = sparse_matrix.fillna(0)
sparse_matrix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 497 entries, 0 to 496
Columns: 444 entries, 0 to 443
dtypes: float64(444)
memory usage: 1.7 MB


In [10]:
# sparse_matrix_sparse = csr_matrix(sparse_matrix.values)
# sparse_matrix_sparse

NameError: name 'csr_matrix' is not defined

In [8]:
# The implicit library expects data as a item-user matrix so we
# create two matricies, one for fitting the model (item-user) 
# and one for recommendations (user-item)
sparse_item_user = sparse.csr_matrix((train_full['like'].astype(float), (train_full['item_id'], train_full['user_id'])))
sparse_user_item = sparse.csr_matrix((train_full['like'].astype(float), (train_full['user_id'], train_full['item_id'])))

In [9]:
sparse_user_item

<497x444 sparse matrix of type '<class 'numpy.float64'>'
	with 8674 stored elements in Compressed Sparse Row format>

### Model

In [13]:
# Initialize the als model and fit it using the sparse item-user matrix
model = implicit.als.AlternatingLeastSquares(factors=1, regularization=0.01, iterations=40)

# Calculate the confidence by multiplying it by our alpha value.
alpha_val = 15
data_conf = (sparse_item_user * alpha_val).astype('double')

#Fit the model
model.fit(data_conf)  



### Find Similar Items

In [91]:
#---------------------
# FIND SIMILAR ITEMS
#---------------------

# Find the 10 most similar
item_id = 400
n_similar = 20

# Use implicit to get similar items.
similar = model.similar_items(item_id, n_similar)

# Print the names of our most similar artists
for item in similar:
    idx, score = item
    print(train_full.item_id.loc[train_full.item_id == idx].iloc[0])


400
304
382
292
287
308
290
286
396
330
406
393
440
428
441
379
288
327
244
340


### Recommendations

In [14]:
#------------------------------
# CREATE USER RECOMMENDATIONS
#------------------------------

recommendations = {}

# Create recommendations for user
for u in test['user_id']:
# for u in range(2):
    
    user_id = u

    # Use the implicit recommender.
    recommended = model.recommend(user_id, sparse_user_item, 20)

    item_id = []
    scores = []

    # Get artist names from ids
    for item in recommended:
        idx, score = item
        item_id.append(train_full.item_id.loc[train_full.item_id == idx].iloc[0])
        scores.append(score)

    # Create a dataframe of artist names and scores
    # recommendations = pd.DataFrame({'user_id': user_id, 'item_id': item_id, 'score': scores})
    recommendations.update({user_id: item_id})


rec_df = pd.DataFrame.from_dict(recommendations)
    
print(rec_df.T)


      0   1   2   3   4   5   6   7   8   9   10   11   12  13  14   15   16  \
166  76  22  35  72  40  80  37  58  65  66   11   32   21   5  60   87  146   
26   76  22  35  72  40  80  37  58  65  66   11   67   32  21   5   60   87   
41   76  22  35  72  40  80  37  58  65  66   11   67   32  21   5   60   87   
286  76  22  35  72  80  37  58  65  66  11   67   32   21   5  60   87  146   
108  76  22  35  72  40  80  37  58  66  11   67   32   21   5  60   87  146   
..   ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ...  ...  ..  ..  ...  ...   
190  76  35  37  65  66  32  21   5  60  87  146  119   33  36  70   30  147   
181  76  22  35  72  40  80  37  58  65  66   11   67   32  21   5   60  146   
448  76  22  35  72  37  58  65  66  67  32   21   60   87   7  33   17   36   
124  76  22  35  72  40  80  58  65  66  11   67   32   21   5  87  146    7   
167  22  40  80  37  58  66  11  21   5  60   87  146  119  78  33  172  147   

      17   18   19  
166    7   44  119

In [15]:
# Экспорт предсказаний в csv
pd.DataFrame(rec_df.T).to_csv('input/out.csv', index=True)